In [1]:
from file_storage import FileStorage
from collections import defaultdict
import tqdm

In [2]:
storage = FileStorage('../storage')

In [3]:
filtered_storage = FileStorage('../filtered_storage')

Отлифльтруем сторадж, полученный в прошлом задании

In [4]:
beginning = 'https://simple.wikipedia.org/wiki/'
ban_patterns = [
    'Help', 'Help_talk', 'Wikipedia', 'Special', 'File', 'Template', 'Talk',
    'Template_talk', 'User_talk', 'User', 'Meta', 'user', 'MediaWiki', 'MediaWiki_talk',
    'Wikipedia_talk', 'Category_talk', 'Module', 'Media', 'Category', 'T'
]

def filter_url(url):
    url_end = url[len(beginning):]
    return any(url_end.startswith(ban_pattern + ':') for ban_pattern in ban_patterns)

In [5]:
for url, page in storage.items():
    if not filter_url(url):
        filtered_storage.write(url, page)

Сторим соседей для каждого урла: извлекаем другие урлы из html, применяем к ним urldefrag и проверяем, что url находится в storage, то есть принадлежит к нашему графу.

In [5]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.parse import urldefrag
import pickle

def extract_links_from_html(url, html):
    parser = BeautifulSoup(html)
    return [
        urldefrag(urljoin(url, link.get('href'))).url
        for link in parser.findAll('a')
    ]

In [6]:
def build_neighbors(storage):
    result = {}
    for url, page in tqdm.tqdm(storage.items()):
        result[url] = [
            link for link in extract_links_from_html(url, page)
            if link in storage
        ]
    return result

In [10]:
neighbors = build_neighbors(filtered_storage)

151127it [1:00:03, 41.94it/s]


In [96]:
with open('neighbors.pkl', 'wb') as f_out:
    pickle.dump(neighbors, f_out)

In [7]:
with open('neighbors.pkl', 'rb') as nei_file:
    neighbors = pickle.load(nei_file, encoding='utf-8')

Вычислим pagerank

In [47]:
def normilize_pagerank(pagerank, normalization_func=max):
    normalization = normalization_func(pagerank.values())
    return {url: rank / normalization for url, rank in pagerank.items()}


def calc_diff(rank_a, rank_b):
    return sum(
        (value_a - rank_b[key]) ** 2
        for key, value_a in rank_a.items()
    ) / len(rank_a)


def calc_pagerank(neighbors, iterations=20, delta=0.1, prev_pagerank=None, min_diff=1e-20):
    if prev_pagerank is None:
        prev_pagerank = defaultdict(lambda: 1 / len(neighbors))

    for ind in range(iterations):
        print('{} iteration, India rank: {}'.format(
            ind, prev_pagerank['https://simple.wikipedia.org/wiki/India']
        ))
        mean_pagerank = sum(prev_pagerank.values()) / len(prev_pagerank)
        pagerank = defaultdict(lambda: mean_pagerank * delta)
        for url in neighbors.keys():
            curr_neighbor_list = neighbors[url]
            for neighbor_url in curr_neighbor_list:
                pagerank[neighbor_url] += prev_pagerank[url] / len(curr_neighbor_list) * (1 - delta)
        pagerank = normilize_pagerank(pagerank)
        diff = calc_diff(pagerank, prev_pagerank)
        print('diff: ' + str(diff))
        if diff < min_diff:
            break
        prev_pagerank = pagerank
    return pagerank

In [48]:
pagerank = calc_pagerank(neighbors, iterations=50)

0 iteration, India rank: 6.6172578083642136e-06
diff: 6.764522103630347e-06
1 iteration, India rank: 0.011445711426637703
diff: 1.0123387660172889e-07
2 iteration, India rank: 0.016829713668104294
diff: 1.9090756872731438e-08
3 iteration, India rank: 0.02009712106461585
diff: 5.363637011027465e-09
4 iteration, India rank: 0.02117589344092674
diff: 1.8901437045676403e-09
5 iteration, India rank: 0.021458291080219623
diff: 7.481314135671983e-10
6 iteration, India rank: 0.021508919015892906
diff: 3.20876343908804e-10
7 iteration, India rank: 0.021497834737330655
diff: 1.490318463878671e-10
8 iteration, India rank: 0.021475544204419692
diff: 7.271933944216867e-11
9 iteration, India rank: 0.02145536312222906
diff: 3.767701491246187e-11
10 iteration, India rank: 0.021439923856564742
diff: 2.0114217405622024e-11
11 iteration, India rank: 0.02142860306202745
diff: 1.1209671267391326e-11
12 iteration, India rank: 0.021420377951550183
diff: 6.357524599216437e-12
13 iteration, India rank: 0.02141

Отсортируем и сохраним результат

In [49]:
sorted_pagerank = sorted(pagerank.items(), key=lambda x: -x[1])

In [50]:
for ind, (url, rank) in enumerate(sorted_pagerank):
    print("{:>3} {:>5.4} {}".format(ind, rank, url))
    if ind == 19:
        break

  0   1.0 https://simple.wikipedia.org/wiki/Main_Page
  1 0.1281 https://simple.wikipedia.org/wiki/International_Standard_Book_Number
  2 0.0725 https://simple.wikipedia.org/wiki/United_States
  3 0.0446 https://simple.wikipedia.org/wiki/France
  4 0.03878 https://simple.wikipedia.org/wiki/Country
  5 0.03762 https://simple.wikipedia.org/wiki/Definition
  6 0.03403 https://simple.wikipedia.org/wiki/United_Kingdom
  7 0.03374 https://simple.wikipedia.org/wiki/Japan
  8 0.03302 https://simple.wikipedia.org/wiki/List_of_Wikipedias
  9 0.03126 https://simple.wikipedia.org/wiki/Medicine
 10 0.03096 https://simple.wikipedia.org/wiki/Art
 11 0.03068 https://simple.wikipedia.org/wiki/Germany
 12 0.03045 https://simple.wikipedia.org/wiki/English_language
 13 0.02833 https://simple.wikipedia.org/wiki/Music
 14 0.0268 https://simple.wikipedia.org/wiki/Europe
 15 0.02547 https://simple.wikipedia.org/wiki/Digital_object_identifier
 16 0.02469 https://simple.wikipedia.org/wiki/Government
 17 0.02429

In [60]:
def dump_sorted_pagerank(sorted_pagerank, path='pagerank_results.txt'):
    with open(path, 'w') as pagerank_file:
        for ind, (url, rank) in enumerate(sorted_pagerank):
            pagerank_file.write("{:>6} {:>7.6} {}\n".format(ind + 1, rank, url))


def load_pagerank(path='pagerank_results.txt'):
    with open(path) as pagerank_file:
        pagerank = {}
        for line in pagerank_file:
            _, rank, url = line.strip().split()
            pagerank[url] = float(rank)
        return pagerank

In [61]:
dump_sorted_pagerank(sorted_pagerank)